In [172]:
import sqlite3
import pandas as pd
import numpy as np
from tqdm import tqdm
import regex as re

import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')

In [141]:
db = sqlite3.connect(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\nlp_annotator-dev\app\db.sqlite3")

## Tables of Use

### Data to Feed into Model
1. gui_alwaysregex to get all the always patterns
2. gui_sentenceannotation to get labels
3. gui_sentence to get sentences
4. gui_sentence_always_regex to get all sentences that matched with always patterns (gives extra 1284 sentences)

### EDA Purposes
1. gui_sentencealwaysregex to see which always expressions get flagged most often
2. gui_seedregex to get list of matched phrases
2. gui_sentenceseedregex to see which expressions in general get flagged most often

## Extracting Data for Model

In [142]:
always_patterns = pd.read_sql_query("Select * from gui_alwaysregex", db)
always_patterns

,id,Pattern,Annotation,CreatedBy_id
0,2,MOCA.*22/30,Y,3
1,3,has memory difficulties,Y,3
2,4,poor.*working memory,Y,3
3,5,MOCA.*28/30,Y,3
4,6,Reports.*short.term.*memory,Y,3
5,7,IMPRESSION.*Memory loss,Y,3
6,8,cognitive deficits,Y,3
7,10,MOCA.*13/30,Y,3
8,12,No\s*memory\s*concerns,N,3
9,13,signs.*suspicious.*dementia,Y,3


In [143]:
labels = pd.read_sql_query("Select * from gui_sentenceannotation", db)
#labels = labels.sort_values(["Sentence_id"])
labels

,id,Label,Annotator_id,Sentence_id
0,1,NTR,3,2990
1,2,YES,3,3003
2,3,YES,3,3009
3,4,NTR,3,3013
4,5,NTR,3,3035
5,6,NTR,3,3045
6,7,NTR,3,3046
7,8,NTR,3,3051
8,9,NTR,3,3054
9,10,NTR,3,3055


In [144]:
matches = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\MASTER_7_14.csv")

C:\Users\MIND_DS\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [145]:
sentences = pd.read_sql_query("Select * from gui_sentence", db)
sentences

,id,Contents,Note_id
0,1,"""ession alone in the meta-analysis. We discuss...",1
1,2,""" ------- sis over her R bra strap, which was ...",2
2,3,""" ------- l obstruction ASSOCIATED DIAGNOSES S...",3
3,4,""" ------- (156 lb) 09/19/17 71.4 kg (157 lb 6...",4
4,5,"""ut answer. Optho notes from recent outpatient...",5
...,...,...,...
279219,279220,"""''she took a break' and wants to discuss with...",274442
279220,279221,"""'all of her medications, including aspirin. D...",274443
279221,279222,"""'Patient, Chart, daughter Olga Mode of contac...",274444
279222,279223,"""'Patient, Chart, daughter Olga Mode of contac...",274445


In [146]:
sentences["Contents"] = matches["padded_merged_regex_sent"]

In [147]:
sentences[sentences['Contents'].str.len() < 700]

,id,Contents,Note_id
23,24,----------------------------------------------...,24
38,39,----------------------------------------------...,39
40,41,----------------------------------------------...,41
56,57,----------------------------------------------...,57
58,59,----------------------------------------------...,59
...,...,...,...
238814,238815,----------------------------------------------...,234037
238824,238825,----------------------------------------------...,234047
238826,238827,----------------------------------------------...,234049
238867,238868,----------------------------------------------...,234090


In [161]:
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\EDA\Getting_Data\keywords.csv")
k = regex["REGEX"].to_list()
c = regex["CASE"].to_list()
p_list = []

for i in range(len(k)):
    if (c[i] == 0):
        p_list.append(re.compile(k[i][5:], re.IGNORECASE))
    elif (c[i] == 1):
        p_list.append(re.compile(k[i]))
print(p_list)

[regex.Regex('\\bdementia\\b', flags=regex.I | regex.V0), regex.Regex('\\bcognition\\b', flags=regex.I | regex.V0), regex.Regex('\\bmemory\\b', flags=regex.I | regex.V0), regex.Regex('\\bMMSE\\b', flags=regex.I | regex.V0), regex.Regex('\\bMOCA\\b', flags=regex.I | regex.V0), regex.Regex('\\bAlzheimer\\b', flags=regex.I | regex.V0), regex.Regex('\\bcognitive\\s*impairment\\b', flags=regex.I | regex.V0), regex.Regex('(?-i:)\\bMCI\\b', flags=regex.V0), regex.Regex('\\bcerebellar\\b', flags=regex.I | regex.V0), regex.Regex('\\bNeurocognitive\\b', flags=regex.I | regex.V0), regex.Regex('\\blewy\\b', flags=regex.I | regex.V0), regex.Regex("\\bpick's\\b", flags=regex.I | regex.V0), regex.Regex('\\bCorticobasal\\b', flags=regex.I | regex.V0), regex.Regex('\\bcerebral\\b', flags=regex.I | regex.V0), regex.Regex('\\bcerebrovascular\\b', flags=regex.I | regex.V0), regex.Regex('\\bamnesia\\b', flags=regex.I | regex.V0), regex.Regex('(?-i:)\\bAD\\b', flags=regex.V0), regex.Regex('(?-i:)\\bLBD\\b',

In [162]:
l = []
for note in tqdm(sentences["Contents"]):
    curr = []
    for p in (p_list):
        m = list(set(re.findall(p, note)))
        m = list(set(map(str.lower, m)))
        if (m != []):
            curr.append("".join(m))
    #print(curr)
    #print(l)
    l.append(str(curr))
sentences["regex_match"] = l

100%|████████████████████████████████████████████████████████████████████████| 279224/279224 [01:04<00:00, 4326.28it/s]


In [148]:
always_pattern_sentences = pd.read_sql_query("Select * from gui_sentencealwaysregex", db)
always_pattern_sentences

,id,AlwaysRegex_id,Sentence_id
0,53,2,377
1,54,2,20965
2,55,2,532
3,56,2,29481
4,57,2,2909
...,...,...,...
8628,9618,37,176918
8629,9619,37,184583
8630,9620,37,248788
8631,9621,38,3086


## Constructing Model Matrix

In [166]:
note = pd.read_sql_query("Select * from gui_note", db)
note

,id,PatientEncounterID,NoteID,Date,PatientID
0,1,3242118020.0,2370049295,2019-03-14,Z6352398
1,2,3181499472.0,1916263837,2018-03-28,Z6352922
2,3,3018789480.0,267681869,2015-01-09,Z6353136
3,4,3172504002.0,1682366001,2017-10-03,Z6353461
4,5,3165982636.0,1853759390,2018-02-07,Z6353764
...,...,...,...,...,...
274441,274442,3285425453.0,3960511043,2020-04-28,Z16320629
274442,274443,3305416153.0,4585859437,2020-09-22,Z16320629
274443,274444,3327429844.0,5203067949,2021-01-25,Z16320629
274444,274445,3349754670.0,5673786544,2021-04-20,Z16320629


In [167]:
print(note.loc[note['id'] == 8481]['PatientID'])

8480    Z6448542
Name: PatientID, dtype: object


In [168]:
def clean_sequence(seq):
    #getting rid of special characters
    seq_no_special_chars = ""
    for character in seq:
        if character.isalnum():
            seq_no_special_chars += character
        elif character == " ":
            seq_no_special_chars += character
            
    #getting rid of numbers
#     seq_no_nums = ""
#     for character in seq_no_special_chars:
#         if not character.isdigit():
#             seq_no_nums += character
            
    #having only 1 space between words
    n = 1
    seq_no_spaces = (' '*n).join(seq_no_special_chars.split())
    
    #Stemming words
#     stemmer = PorterStemmer()
#     tokens = word_tokenize(seq_no_spaces)
#     sequences_clean = []
#     for word in tokens:
#         if word != " ":
#             stem_word = stemmer.stem(word)  # stemming word
#         sequences_clean.append(stem_word)
    
#     final = " "
#     final = final.join(sequences_clean)
    
    return seq_no_spaces.lower()

In [169]:
print(sentences["Contents"][1234], "\n")
print(clean_sequence(sentences["Contents"][1234]))

 ------- on Evaluation Impression: John V Graziano is a 91 y.o. male who presents to PT with primary physical therapy impairments in ROM, strength and pain most significantly in R LE in setting of likely R femoral neck fracture (unclear on imaging) as well as impairments in mental status and cognition at this time all of which are limiting his tolerance and participation in PT session. However per r ------- et stronger again Precautions: NWB R LE per ortho, bedrest (clarifying with team) Arousal/Attention/Cognition/Communication: Opens eyes to tactile and verbal cues and regards examiner briefly says hello and inconsistently follows very simple commands today. Communicates pain and localizes by pointing today Pain: Reports yes to pain and points to R hip - does not report a number for pain Ran 

on evaluation impression john v graziano is a 91 yo male who presents to pt with primary physical therapy impairments in rom strength and pain most significantly in r le in setting of likely r 

In [203]:
#getting sentence ids
sentence_id = labels["Sentence_id"].to_list()
#print(len(sentence_id))
sentence_id += always_pattern_sentences["Sentence_id"].to_list()
#print(len(sentence_id))

# defining model matrix and setting dimensions
model_matrix = pd.DataFrame(columns = ["patient_id", "sequence", "original", "label"])
model_matrix["patient_id"] = [0] * int(len(sentence_id))
model_matrix["sequence"] = [" "] * int(len(sentence_id))
model_matrix["original"] = [" "] * int(len(sentence_id))
model_matrix["label"] = [" "] * int(len(sentence_id))

In [184]:
sentences

,id,Contents,Note_id,regex_match
0,1,ked at HMS in the department of psychobiology ...,1,['cerebral']
1,2,or lymphadenopathy. Chest: Clear to auscultati...,2,['memory']
2,3,ession alone in the meta-analysis. We discusse...,3,['dementia']
3,4,------- ing tylenol really accept for this we...,4,"['cognition', 'memory', 'moca']"
4,5,------- in is related to cancer. Reassuringly...,5,"['cognition', 'memory']"
...,...,...,...,...
279219,279220,"------- se ""she took a break"" and wants to dis...",274442,"['cognition', 'memory', 'cerebellar', 'cerebral']"
279220,279221,"------- all of her medications, including asp...",274443,"['cognition', 'memory', 'cerebellar', 'cerebral']"
279221,279222,"------- ? Patient, Chart, daughter Olga Mode o...",274444,"['dementia', 'cognition', 'memory', 'mci', 'ce..."
279222,279223,"------- ? Patient, Chart, daughter Olga Mode o...",274445,"['dementia', 'cognition', 'moca', 'cognitive i..."


In [204]:
#extracting necessary data
idx = 0
for i in range(int(len(labels["Sentence_id"].to_list()))):
    #if (i < len(labels["Sentence_id"].to_list())):
    # getting all info from sentences that were manually annotated
    curr_note_id = sentences[sentences['id'] == sentence_id[i]]['Note_id'].values
    curr_patient_id = note.loc[note['id'] == curr_note_id[0]]['PatientID'].values
    curr_seq = sentences[sentences['Note_id'] == sentence_id[i]]['Contents'].values
    curr_label = labels[labels["Sentence_id"] == sentence_id[i]]['Label'].values
    original = curr_seq[0]
    curr_seq[0] = clean_sequence(curr_seq[0])

    model_matrix.iloc[i] = (curr_patient_id[0], curr_seq[0], original, curr_label[0])

    idx += 1

#print(idx) 

j = 1    
while (idx < len(sentence_id)): 
    #getting info from sentences that got auto annotated through always pattern matches
    #curr_sentence_id = always_pattern_sentences[always_pattern_sentences['id'] == j]['Sentence_id'].values
    curr_seq = sentences[sentences['id'] == sentence_id[idx]]['Contents'].values
    #print(curr_seq[0])
    curr_note_id = sentences[sentences['id'] == sentence_id[idx]]['Note_id'].values
    curr_patient_id = note.loc[note['id'] == curr_note_id[0]]['PatientID'].values
    #print(curr_patient_id)
    curr_always_regex_id = always_pattern_sentences[always_pattern_sentences['Sentence_id'] == sentence_id[idx]]['AlwaysRegex_id'].values
    #print(curr_always_regex_id)
    curr_label = always_patterns[always_patterns['id'] == curr_always_regex_id[0]]['Annotation'].values
    original = curr_seq[0]
    curr_seq[0] = clean_sequence(curr_seq[0])

    model_matrix.iloc[idx] = (curr_patient_id[0], curr_seq[0], original, curr_label[0])

    j += 1
    idx += 1

# yes = model_matrix[model_matrix['label'] == 3]
# neither = model_matrix[model_matrix['label'] == 2]
# no = model_matrix[model_matrix['label'] == 1]

# yes = yes.to_csv("yes.csv")
# neither = neither.to_csv("neither.csv")
# no = no.to_csv("no.csv"

In [205]:
model_matrix

,patient_id,sequence,original,label
0,Z12144959,colleen a monaghan md maria isabel ajanel zieg...,----------------------------------------------...,NTR
1,Z6411816,edness no syncope no dyspnea endocrine no poly...,"edness, No Syncope, No Dyspnea Endocrine No Po...",YES
2,Z6631913,y from india visits there at least once per ye...,"y from India, visits there at least once per y...",YES
3,Z6755752,03 uncoded surgical history mastectomy bilater...,------- 03 • Uncoded surgical history Mastect...,NTR
4,Z7854395,er violence fear of current or ex partner not ...,------- er violence Fear of current or ex par...,NTR
...,...,...,...,...
8651,Z11313827,ening 1 urinary frequency with a history of bp...,ening) 1. Urinary frequency with a history of ...,Y
8652,Z8098120,moca 1130 january 2016 certainly of vascular e...,----------------------------------------------...,Y
8653,Z9368658,lan refer to severe episode of recurrent major...,------- lan: Refer to severe episode of recurr...,Y
8654,Z9778741,ast carcinoma hpi claire took letrozole for a ...,ast carcinoma.. HPI Claire took letrozole for ...,Y


In [206]:
#converting yes,no,ntr labels to numerical equivalents
mappings = {"NO" : 0, "NTR" : 1, "YES" : 2, "N": 0, "T": 1, "Y": 2}
model_matrix.label = [mappings[item] for item in model_matrix.label]

model_matrix = model_matrix.sample(frac=1).reset_index(drop=True) #shuffling model_matrix

In [207]:
model_matrix[model_matrix["sequence"].str.len() < 500]

,patient_id,sequence,original,label
38,Z7193519,pt continues to feel foggy about his memory ao...,----------------------------------------------...,0
61,Z10380050,hi just fyi moca cw significant global impairm...,----------------------------------------------...,2
165,Z8826699,s patient reports some difficulties with short...,----------------------------------------------...,2
256,Z6836637,continues to be an issue also with family hx o...,----------------------------------------------...,1
259,Z9122254,no recurrence after single acute confusional e...,----------------------------------------------...,1
...,...,...,...,...
8455,Z8049522,she is not had any interval strokes or tia sym...,----------------------------------------------...,0
8482,Z12359379,spoke with patients outpatient neurologist dr ...,----------------------------------------------...,2
8484,Z8409839,pt has felt no improvement since beginning tx ...,----------------------------------------------...,0
8535,Z10144902,patient reports continual word finding difficu...,----------------------------------------------...,2


In [208]:
model_matrix.to_csv(r"../Modeling/input.csv")

## Verification of Results

In [32]:
clean_seq = clean_sequence(str(sentences[sentences['Note_id'] == sentence_id[0]]['Contents'].values[0]))
model_matrix[model_matrix['sequence'] == clean_seq]

,patient_id,sequence,label,len
2324,Z12144959,phenomenology of sx and pt preference she woul...,1,752


In [31]:
s_id = sentences[sentences['Contents'] == str(sentences[sentences['Note_id'] == sentence_id[0]]['Contents'].values[0])]['id'].values[0]
labels[labels['Sentence_id'] == s_id]['Label'].values[0]

'NTR'

In [62]:
#exporting data to csv's
model_matrix.to_csv("../Modeling/Data/input.csv")

## EDA

In [ ]:
regex_phrases = pd.read_sql_query("Select * from gui_seedregex", db)
phrase_matches = pd.read_sql_query("Select * from gui_sentenceseedregex", db)
freq = phrase_matches['SeedRegex_id'].value_counts().to_dict()
counter = 1
for key, value in freq.items():
    print("Match #", counter)
    print("Id: ", key)
    print("Pattern:", regex_phrases[regex_phrases['id'] == key]['Pattern'].values)
    print("Count: ", value, "\n")
    counter += 1

In [ ]:
# Printing out frequencies of always pattern matches 
always_pattern_matches = pd.read_sql_query("Select * from gui_sentencealwaysregex", db)
freq = always_pattern_matches['AlwaysRegex_id'].value_counts().to_dict()
counter = 1
for key, value in freq.items():
    print("Match #", counter)
    print("Id: ", key)
    print("Pattern:", always_patterns[always_patterns['id'] == key]['Pattern'].values)
    print("Count: ", value, "\n")
    counter += 1